# Pré Processamento de Dados e Pipelines

Este notebook visa abordar algumas etapas usuais de limpeza de dados, seja para análises estatísticas dos dados ou preparar os dados para servirem de input para a maioria dos modelos de Machine Learning!

A limpeza e preprocessamento de dados contém diversas etapas como:
 - Remoção ou substituição de dados faltantes
 - Criação de dummies para as variáveis categóricas
 - Padronização de dados numéricos
 - Remoção ou substituição de outliers
 - Principal Component Analysis

Abordaremos estas transformações em um dataset de aplicações de crédito e por fim iremos criar uma pipeline com métodos nativos do Scikit-learn que automatize as transformações para melhorar o fluxo e processamento dos dados, preparando-os para uma futura aplicação.

### Bibliotecas

In [55]:
import pandas as pd
import numpy as np

from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler

### Importação do Dataset

In [35]:
df = pd.read_feather('credit_scoring.ftr')

# Abandonando a coluna de datas, indice e a variável alvo:

df.drop(columns=['data_ref', 'mau', 'index'], inplace=True)

# Amostra dos dados

df.sample(n=5)

,sexo,posse_de_veiculo,posse_de_imovel,qtd_filhos,tipo_renda,educacao,estado_civil,tipo_residencia,idade,tempo_emprego,qt_pessoas_residencia,renda
445197,M,S,N,0,Assalariado,Superior incompleto,Casado,Com os pais,24,2.630137,2.0,4497.79
612193,F,N,S,0,Servidor público,Superior completo,Solteiro,Casa,46,0.931507,1.0,714.65
306923,F,N,N,0,Pensionista,Superior completo,Casado,Casa,59,NaN,2.0,21784.79
164749,F,N,S,0,Pensionista,Médio,Casado,Casa,57,NaN,2.0,9886.86
59735,F,N,S,1,Assalariado,Médio,Casado,Casa,27,8.684932,3.0,2334.84


In [36]:
# Podemos construir uma tabela com as informações do dataset, para nos auxiliar na limpeza:

metadados = pd.DataFrame({'tipo':df.dtypes})
metadados['valores_unicos'] = df.nunique()
metadados['faltantes_pct'] = round(df.isnull().sum()/df.shape[0], 3)
metadados['faltantes'] = df.isnull().sum()

metadados

,tipo,valores_unicos,faltantes_pct,faltantes
sexo,object,2,0.000,0
posse_de_veiculo,object,2,0.000,0
posse_de_imovel,object,2,0.000,0
qtd_filhos,int64,8,0.000,0
tipo_renda,object,5,0.000,0
educacao,object,5,0.000,0
estado_civil,object,5,0.000,0
tipo_residencia,object,6,0.000,0
idade,int64,47,0.000,0
tempo_emprego,float64,3004,0.168,125957


In [37]:
# Listando as colunas com variáveis numéricas e categóricas:

num = list(df.select_dtypes('number').columns)

cat = list(df.select_dtypes(exclude='number').columns)

print(f'Número de variáveis numéricas: {len(num)} \n{num}')

print(f'\nNúmero de variáveis categóricas: {len(cat)} \n{cat}')

Número de variáveis numéricas: 5 
['qtd_filhos', 'idade', 'tempo_emprego', 'qt_pessoas_residencia', 'renda']

Número de variáveis categóricas: 7 
['sexo', 'posse_de_veiculo', 'posse_de_imovel', 'tipo_renda', 'educacao', 'estado_civil', 'tipo_residencia']


A partir destas informações podemos direcionar quais métodos utilizaremos para limpar os dados.

### Substituindo valores faltantes

Caso tenhamos uma grande quantidade de dados, podemos descartar dados faltantes sem grandes perdas. Caso a porcentagem de dados faltantes seja muito grande ou nosso dataset seja pequeno, excluir linhas pode acabar impactando a generalidade da variável, levando a análises errôneas e menor precisão de modelos preditivos.

Uma estratégia mais interessante é substituir os dados faltantes, seja pela média para variáveis numéricas ou seja pelo valor mais frequênte para variáveis categóricas.

O dataset acima apresenta dados faltantes apenas para a variável 'tempo_emprego', porém vamos tratar métodos gerais, para variáveis numéricas e categóricas.

In [38]:
# Podemos substituir diretamente o valor numérico faltante de uma variável por sua média:

df.fillna(value=df['tempo_emprego'].mean(), inplace=True)

# O mesmo pode ser feito para variáveis categóricas, ou podemos utilizar um método mais
# sofisticado, construíndo uma função que substitua valores faltantes seguindo a proporção
# dos valores da variável:

def preencher_proporção(col):
    """ Preenche valores ausentes na mesma proporção dos valores presentes

    Recebe uma coluna e retorna a coluna com os valores faltantes preenchidos
    na proporção dos valores existentes."""

    # gerando o dicionário com valores únicos e sua porcentagem:
    por = col.value_counts(normalize=True).to_dict()

    # transformando as chaves e valores do dicionário em uma lista:
    percent = [por[key] for key in por]
    lab = [key for key in por]

    # utilizando as listas para preencher os valores nulos na proporção correta:
    s = pd.Series(np.random.choice(lab, p=percent, size=col.isnull().sum()))
    col = col.fillna(s)

    # verificando se todos os valores ausentes foram preenchidos e
    # preenchendo os que não tiverem sido:
    if len(col.isnull()) > 0:
        col.fillna(value=max(por, key=por.get), inplace=True, axis=0)
    
    return col

In [39]:
# Para remover todos os dados faltantes, iteramos sobre todas colunas do dataset,
# utilizando a função acima para variáveis categóricas e substituindo pela média 
# para variáveis numéricas:

for var in df.columns:

    if df[var].dtypes == 'O':
        df[var] = preencher_proporção(df[var])

    else:
        df[var] = df[var].fillna(value=df[var].mean(), axis=0)

# Visualizando a existência de dados faltantes:

df.isnull().sum()

sexo                     0
posse_de_veiculo         0
posse_de_imovel          0
qtd_filhos               0
tipo_renda               0
educacao                 0
estado_civil             0
tipo_residencia          0
idade                    0
tempo_emprego            0
qt_pessoas_residencia    0
renda                    0
dtype: int64

### Dummies

é comum a existência de variáveis categóricas em datasets. Um problema comum enfrentado por iniciantes em ciência de dados é tratar estas variáveis, uma vez que os modelos de Machine Learning mais usuais como Regressões, K-Means e SVM não aceitam estes tipos de dados.

Uma estratégia possível é a substituição das classes da variável por um valor numérico. Este tratamento é mais adequado quando existe uma certa hierarquia entre as classes. 

Caso não identifiquemos esta relação, podemos criar dummies das variáveis, isto é, criar colunas com flags de cada categoria da variável.

A criação de dummies pode ser uma ótima alternativa para modelos preditivos, porém se a variável ter muitos valores únicos, muitas colunas serão criadas, aumentando assim a dimensão do dataset, te levando à *maldição da dimensionalidade*.

In [40]:
# Criação de dummies:

df_dum = pd.get_dummies(df, drop_first=True).astype(int)

df_dum.head()

,qtd_filhos,idade,tempo_emprego,qt_pessoas_residencia,renda,sexo_M,posse_de_veiculo_S,posse_de_imovel_S,tipo_renda_Bolsista,tipo_renda_Empresário,...,educacao_Superior incompleto,estado_civil_Separado,estado_civil_Solteiro,estado_civil_União,estado_civil_Viúvo,tipo_residencia_Casa,tipo_residencia_Com os pais,tipo_residencia_Comunitário,tipo_residencia_Estúdio,tipo_residencia_Governamental
0,0,43,6,1,2515,0,0,0,0,1,...,0,0,1,0,0,1,0,0,0,0
1,0,35,4,2,3180,0,1,1,0,0,...,0,0,0,0,0,1,0,0,0,0
2,2,31,0,4,1582,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
3,0,54,12,2,13721,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
4,0,31,8,1,2891,0,1,0,0,0,...,1,0,1,0,0,1,0,0,0,0


Mesmo nosso dataset contendo variáveis categóricas com poucos valores únicos, a dimensão do dataset aumentou consideravelmente!

In [41]:
print(f'Dimensão do dataset inicial: {df.shape[1]}\nDimensão do dataset com dummies: {df_dum.shape[1]}')

Dimensão do dataset inicial: 12
Dimensão do dataset com dummies: 25


O problema da maldição da dimensionalidade pode ser resolvido fazendo uma melhor selação de variáveis ou realizando uma PCA.

### Padronização dos dados

A padronização dos dados numéricos consiste em aplicar uma transformação nestes dados, fazendo com que a distribuição dos valores tenha média 0 e desvio igual a 1. Este tratamento é especialmente útil em clusterizações, visto que os modelos podem encontrar dificuldade para fazer agrupamentos corretos tendo dados numéricos com escalas diferentes devido a definição de distância usada pelos modelos.

A padronização (z) de uma variável *X* é definida como:

$z=\dfrac{x-\mu}{s}$

Onde $\mu$ é a média e $s$ é o desvio padrão.

In [42]:
# Para aplicar a padronização, precisamos criar uma função para realizar esta transformação:

def padronizacao(var):

    media = df[var].mean()
    desvio = df[var].std()

    df[var] = df[var].apply(lambda x: (x-media))

    return df[var]

In [43]:
# Iterando sobre todas as variáveis numéricas:

for var in num:
    df_dum[var] = padronizacao(var)

# Dados numéricos com a padronização aplicada:

df_dum.head()

,qtd_filhos,idade,tempo_emprego,qt_pessoas_residencia,renda,sexo_M,posse_de_veiculo_S,posse_de_imovel_S,tipo_renda_Bolsista,tipo_renda_Empresário,...,educacao_Superior incompleto,estado_civil_Separado,estado_civil_Solteiro,estado_civil_União,estado_civil_Viúvo,tipo_residencia_Casa,tipo_residencia_Com os pais,tipo_residencia_Comunitário,tipo_residencia_Estúdio,tipo_residencia_Governamental
0,-0.433251,-0.804733,-0.874601,-1.212241,-20723.154747,0,0,0,0,1,...,0,0,1,0,0,1,0,0,0,0
1,-0.433251,-8.804733,-3.222547,-0.212241,-20058.354747,0,1,1,0,0,...,0,0,0,0,0,1,0,0,0,0
2,1.566749,-12.804733,-7.504738,1.787759,-21656.254747,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
3,-0.433251,10.195267,5.024029,-0.212241,-9517.374747,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
4,-0.433251,-12.804733,0.684303,-1.212241,-20347.464747,0,1,0,0,0,...,1,0,1,0,0,1,0,0,0,0


### Substituindo outliers

A presença de outliers pode afetar drasticamente a performance de modelos ou gerar conclusões errôneas em uma análise de dados. Como nos dados faltantes, uma opção é a remoção dos outliers.

Aqui iremos utilizar a distância interquartil para calssificar os outliers e substituir eles pela mediana.

In [44]:
# Definindo as funções:

def outliers_filtro(df, var):
    ''' Substituí os outliers da variável pela mediana '''
    q1 = df[var].quantile(0.25)
    q3 = df[var].quantile(0.75)
    iqr = q3 - q1

    mediana = df[var].median()

    filtro = (df[var] >= q1 - 1.5*iqr) & (df[var] <= q3 + 1.5*iqr)
    #df.loc[~filtro, var] = mediana
    df[var] = np.where(~filtro, mediana, df[var])

    return df

In [45]:
for i in num:
    df_dum[i] = outliers_filtro(df, i)[i]


df_dum.head()

,qtd_filhos,idade,tempo_emprego,qt_pessoas_residencia,renda,sexo_M,posse_de_veiculo_S,posse_de_imovel_S,tipo_renda_Bolsista,tipo_renda_Empresário,...,educacao_Superior incompleto,estado_civil_Separado,estado_civil_Solteiro,estado_civil_União,estado_civil_Viúvo,tipo_residencia_Casa,tipo_residencia_Com os pais,tipo_residencia_Comunitário,tipo_residencia_Estúdio,tipo_residencia_Governamental
0,-0.433251,-0.804733,-0.874601,-1.212241,-20723.154747,0,0,0,0,1,...,0,0,1,0,0,1,0,0,0,0
1,-0.433251,-8.804733,-3.222547,-0.212241,-20058.354747,0,1,1,0,0,...,0,0,0,0,0,1,0,0,0,0
2,1.566749,-12.804733,-7.504738,1.787759,-21656.254747,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
3,-0.433251,10.195267,5.024029,-0.212241,-9517.374747,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
4,-0.433251,-12.804733,0.684303,-1.212241,-20347.464747,0,1,0,0,0,...,1,0,1,0,0,1,0,0,0,0


### Principal component analysis

PCA é um método de redução de dimensionalidade, essa redução é feita por meio de componentes principais, que "agrupam" as informações das variáveis por meio de combinações lineares.

A PCA é importante também para reduzir o tempo de processamento para treinar modelos, possibilitando técnicas como um grid search para tunning de hiperparâmetros em modelos treinados com muitos dados.

In [47]:
# Para realizar essa etapa, vamos utilizar uma ferramenta do Scikit-Learn:

pca = PCA().fit(df_dum)

# Listando a taxa de explicabilidade por número de componentes:

for i in range(1,11):
    print(f'n_comp {i}: {pca.explained_variance_ratio_[:i].sum()}')

n_comp 1: 0.9999980169486306
n_comp 2: 0.9999997960977467
n_comp 3: 0.9999999602721479
n_comp 4: 0.9999999731295682
n_comp 5: 0.9999999785645454
n_comp 6: 0.9999999826813066
n_comp 7: 0.9999999858263889
n_comp 8: 0.9999999883164348
n_comp 9: 0.9999999903107512
n_comp 10: 0.9999999921331206


Com uma única componente da PCA conseguimos explicar mais de 99% da variância dos dados.

In [53]:
pca_array = PCA(n_components=3).fit_transform(df_dum)

# Assim temos os dados prontos para o treinamento na maioria dos modelos de ML.

pca_array

array([[-7.06887703e+03, -1.96207677e-01,  1.24860798e+00],
       [-6.40407786e+03, -8.42030328e+00, -5.08955757e-01],
       [-8.00197865e+03, -1.27175714e+01, -4.20544309e+00],
       ...,
       [-8.93520658e+03,  1.00022805e-01,  5.03340117e+00],
       [-5.36754532e+03,  1.31919214e+01,  6.89928849e+00],
       [-5.12357661e+03, -1.91480445e-01,  4.57980709e+00]])

In [54]:
# Caso seja mais conveniente, podemos ainda manter o formato de dataframe:

df_final = pd.DataFrame(pca_array)

df_final

,0,1,2
0,-7068.877034,-0.196208,1.248608
1,-6404.077864,-8.420303,-0.508956
2,-8001.978654,-12.717571,-4.205443
3,4136.904325,10.357860,4.944502
4,-6693.187643,-12.060371,3.721392
...,...,...,...
749995,-7474.277165,4.465270,-2.915247
749996,-8194.875443,21.900981,0.503615
749997,-8935.206575,0.100023,5.033401
749998,-5367.545318,13.191921,6.899288


Com isso concluímos o exemplo de aplicações de algumas etapas de limpeza e pré processamento de dados. É importante notar que as decisões de quais métodos aplicar é baseada em um entendimento sobre os dados, como dito acima, para certos casos pode ser mais interessante remover outliers e faltantes, ou aplicar uma transformação diferentes, caso desejamos analisar as variáveis realizar um PCA não é interessante e assim por diante.

### Pipelines

Agora que entendemos mais sobre o pré processamento de dados, podemos contruir pipelines que automatizem estes processos. Criar pipeline para os dados é extremamente importante, seja para entregar dados limpos para um analísta de dados criar seus dashboards ou para realizar previsões sobre os dados usando um modelo que esta em produção.

In [56]:
# Recarregando os dados:

df = pd.read_feather('credit_scoring.ftr')
df.drop(columns=['data_ref', 'mau', 'index'], inplace=True)

num = list(df.select_dtypes('number').columns)
cat = list(df.select_dtypes(exclude='number').columns)

df.head()

,sexo,posse_de_veiculo,posse_de_imovel,qtd_filhos,tipo_renda,educacao,estado_civil,tipo_residencia,idade,tempo_emprego,qt_pessoas_residencia,renda
0,F,N,N,0,Empresário,Médio,Solteiro,Casa,43,6.873973,1.0,2515.39
1,F,S,S,0,Assalariado,Médio,Casado,Casa,35,4.526027,2.0,3180.19
2,F,N,N,2,Assalariado,Médio,Casado,Casa,31,0.243836,4.0,1582.29
3,F,N,N,0,Assalariado,Médio,Casado,Casa,54,12.772603,2.0,13721.17
4,F,S,N,0,Assalariado,Superior incompleto,Solteiro,Casa,31,8.432877,1.0,2891.08


In [57]:
# Etapas da pipeline:

# Substituição de dados faltantes:

impt_num = SimpleImputer(strategy='mean')
impt_cat = SimpleImputer(strategy='most_frequent')

# Criação de dummies:

dummies = OneHotEncoder(handle_unknown='ignore', sparse_output=False)

# Padronização:

scaler = StandardScaler()

# Preprocessamento separado por tipo de dado:

prep = ColumnTransformer(
    transformers=[

        ('num', Pipeline(steps=[
            ('imputer', impt_num),
            ('scaler', scaler)
        ]), num),

        ('cat', Pipeline(steps=[
            ('imputer', impt_cat),
            ('encoding', dummies)
        ]), cat)
])

# PCA:

pca = PCA(n_components=3)

In [58]:
# Criação da pipeline:

pipe = Pipeline(steps=[
    ('preprocessor', prep),
    ('pca', pca)
])

pipe

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer()),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['qtd_filhos', 'idade',
                                                   'tempo_emprego',
                                                   'qt_pessoas_residencia',
                                                   'renda']),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('encoding',
                                                                   OneHotEncoder(handle_unknown='ignore',
                                                                                 sparse_output=False))]),
                                                  ['sexo', 'posse_de_veiculo',
                                                   'posse_de_imovel',
                                                   'tipo_renda', 'educacao',
                                                   'estado_civil',
                                                   'tipo_residencia'])])),
                ('pca', PCA(n_components=3))])

Acima podemos ver a estrutura da pipeline e acessar informações de cada passo!

In [61]:
# Fitando e armazenando a transformação em um array:

array_pipe = pipe.fit_transform(df)

# Transformando em dataframe:

df_final = pd.DataFrame(array_pipe)

df_final

,0,1,2
0,-1.235266,-0.853483,0.600671
1,0.111627,-0.738339,0.636795
2,3.159182,0.000104,-0.504746
3,-1.063747,0.410355,-0.505189
4,-0.565267,-0.781580,1.816791
...,...,...,...
749995,-0.304312,-0.889379,-0.029935
749996,-2.291528,-0.525790,-1.321124
749997,2.665308,1.100539,-0.762019
749998,-1.978188,0.136467,-0.477478


Com isso os dados estão prontos para serem usados por um modelo de machine learning!